## NMEA Parsing, in Python

In [1]:
import nmea_parser as NMEAParser

---------------------
nmea_parser NOT running as main
---------------------


In [2]:
samples = [
    "$IIRMC,092551,A,1036.145,S,15621.845,W,04.8,317,,10,E,A*0D\r\n",
    "$IIMWV,088,T,14.34,N,A*27\r\n",
    "$IIVWR,148.,L,02.4,N,01.2,M,04.4,K*XX\r\n",
    "$IIVTG,054.7,T,034.4,M,005.5,N,010.2,K,A*XX\r\n",
    "$GPRMC,183333.000,A,4047.7034,N,07247.9938,W,0.66,196.21,150912,,,A*7C\r\n"
]

In [3]:

# akeu = sex_to_dec("12", "34.XX")

for sentence in samples:
    print("Parsing {}".format(sentence))
    try:
        nmea_obj = NMEAParser.parse_nmea_sentence(sentence)
        print('Parsed Object: {}'.format(nmea_obj))
    except Exception as ex:
        print("Ooops! {}".format(ex))
    print("----------------------------------")


Parsing $IIRMC,092551,A,1036.145,S,15621.845,W,04.8,317,,10,E,A*0D

Parsing RMC, 13 items
Parsed Object: {'type': 'rmc', 'parsed': {'valid': 'true', 'position': {'latitude': -10.602416666666667, 'longitude': -156.10241666666667}}}
----------------------------------
Parsing $IIMWV,088,T,14.34,N,A*27

Ooops! No parser exists for MWV
----------------------------------
Parsing $IIVWR,148.,L,02.4,N,01.2,M,04.4,K*XX

Invalid Hex CS Key XX
Ooops! Invalid checksum
----------------------------------
Parsing $IIVTG,054.7,T,034.4,M,005.5,N,010.2,K,A*XX

Invalid Hex CS Key XX
Ooops! Invalid checksum
----------------------------------
Parsing $GPRMC,183333.000,A,4047.7034,N,07247.9938,W,0.66,196.21,150912,,,A*7C

Parsing RMC, 13 items
Parsed Object: {'type': 'rmc', 'parsed': {'valid': 'true', 'position': {'latitude': 40.79505666666667, 'longitude': -72.12839}}}
----------------------------------
